In [15]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [17]:
# 假設有一個文本數據
corpus = [
    "This is the first document.",
    "This is the second second document.",
    "And the third one.",
    "Is this the first document?",
]

## 詞袋
vectorize_text回傳一個dataframe, 其中每一行都是文檔的向量表示。

In [18]:
def vectorize_text(corpus):
  bag_of_words_model = CountVectorizer()
  # 統計詞頻
  dense_vec_matrix = bag_of_words_model.fit_transform(corpus).todense()

  # 轉換為dataframe
  bag_of_words_df = pd.DataFrame(dense_vec_matrix)

  # 為dataframe添加別名
  bag_of_words_df.columns = sorted(bag_of_words_model.vocabulary_)

  return bag_of_words_df

In [19]:
df = vectorize_text(corpus)
df.head()

,and,document,first,is,one,second,the,third,this
0,0,1,1,1,0,0,1,0,1
1,0,1,0,1,0,2,1,0,1
2,1,0,0,0,1,0,1,1,0
3,0,1,1,1,0,0,1,0,1


## 最大特徵數
通過設置 `max_features` 參數來限制詞袋中的最大特徵數，只保留出現頻率最高的前幾個單詞

```
vectorizer = CountVectorizer(max_features=1000) # 只保留出現頻率最高的前 1000 個單詞
```
取得詞袋中前Ｎ個頻率最高的單字，並建立一個dataframe顯示。

In [20]:
def bow_top_n(corpus, n):
  # 對所有關鍵字的frequency進行排序
  bag_of_words_model_small = CountVectorizer(max_features=n)

  # 統計詞頻轉成dataframe
  bag_of_words_df_small = pd.DataFrame(bag_of_words_model_small.fit_transform(corpus).todense())

  # 加上列名
  bag_of_words_df_small.columns = sorted(bag_of_words_model_small.vocabulary_)

  return bag_of_words_df_small

In [24]:
df_2 = bow_top_n(corpus,3)
df_2.head()

,document,is,the
0,1,1,1
1,1,1,1
2,0,0,1
3,1,1,1


## stop_words

通過指定停用詞列表來過濾在文本中不需要考慮的單詞。比如使用內置的英文停用詞列表
```
vectorizer = CountVectorizer(stop_words='english')
```

1. 如果 stop_words 被設置為 'english'，則會使用內置的英文停用詞列表。然而，這裡指出了 'english' 停用詞列表可能存在一些已知的問題，因此建議考慮使用其他替代方案，可以參考 "Using stop words"（使用停用詞）部分來查找其他選擇。

2. 如果 stop_words 被設置為一個列表，則該列表應包含你希望從文本中過濾掉的停用詞。

3. 如果 stop_words 被設置為 None，則不使用任何停用詞。在這種情況下，你可以考慮調整 max_df 參數的值，例如設置為 (0.7, 1.0) 範圍內的一個較大值，以自動檢測和過濾基於文檔內部語料庫中詞頻的停用詞。

In [25]:
# 自定義停用詞列表
custom_stop_words = ['is', 'the', 'this']

def stop_words(corpus):
  # 濾掉停用詞
  # bag_of_words_model_small = CountVectorizer(stop_words='english')
  bag_of_words_model_small = CountVectorizer(stop_words=custom_stop_words)

  # 統計詞頻轉成dataframe
  bag_of_words_df_small = pd.DataFrame(bag_of_words_model_small.fit_transform(corpus).todense())

  # 加上列名
  bag_of_words_df_small.columns = sorted(bag_of_words_model_small.vocabulary_)

  return bag_of_words_df_small

In [26]:
df_3 = stop_words(corpus)
df_3.head()

,and,document,first,one,second,third
0,0,1,1,0,0,0
1,0,1,0,0,2,0
2,1,0,0,1,0,1
3,0,1,1,0,0,0


## N-gram範圍
通過指定 `ngram_range` 參數來考慮多詞（N-gram）的情況。例如，使用 1 到 3 個連續單詞
```
vectorizer = CountVectorizer(ngram_range=(1, 3))
```

在自然語言處理（NLP）中，N-gram 是由 N 個連續單詞組成的片段。通常，N-gram 是用來捕捉文本中的短語或連續詞組的一種方法。使用 N-gram 可以捕捉單詞之間的語義關聯，並提供更多的上下文信息。

其中第一個整數表示最小的 N 值，第二個整數表示最大的 N 值

在這個示例中，我們指定 ngram_range=(1, 2)，這意味著 CountVectorizer 會生成單詞級別（unigram）和二詞級別（bigram）的 N-gram 特徵。

In [36]:
def ngram_range(corpus, unigram, bigram):
  # 設定連續詞範圍
  bag_of_words_model_small = CountVectorizer(ngram_range=(unigram, bigram))

  # 統計詞頻轉成dataframe
  bag_of_words_df_small = pd.DataFrame(bag_of_words_model_small.fit_transform(corpus).todense())

  # 加上列名
  bag_of_words_df_small.columns = sorted(bag_of_words_model_small.vocabulary_)

  return bag_of_words_df_small

In [34]:
df_4 = ngram_range(corpus, 1, 2)
df_4.head()

,and,and the,document,first,first document,is,is the,is this,one,second,...,second second,the,the first,the second,the third,third,third one,this,this is,this the
0,0,0,1,1,1,1,1,0,0,0,...,0,1,1,0,0,0,0,1,1,0
1,0,0,1,0,0,1,1,0,0,2,...,1,1,0,1,0,0,0,1,1,0
2,1,1,0,0,0,0,0,0,1,0,...,0,1,0,0,1,1,1,0,0,0
3,0,0,1,1,1,1,0,1,0,0,...,0,1,1,0,0,0,0,1,0,1


## 最大和最小文檔頻率
通過設置 `max_df` 和 `min_df` 參數來過濾掉過於常見或過於罕見的單詞。比如只考慮在至少 2 個文檔中出現，但不超過 80% 的文檔中出現的單詞
```
vectorizer = CountVectorizer(min_df=2, max_df=0.8)
```

In [40]:
def df_range(corpus, max_df, min_df):
  # 濾掉停用詞
  bag_of_words_model_small = CountVectorizer(min_df=min_df, max_df=max_df)

  # 統計詞頻轉成dataframe
  bag_of_words_df_small = pd.DataFrame(bag_of_words_model_small.fit_transform(corpus).todense())

  # 加上列名
  bag_of_words_df_small.columns = sorted(bag_of_words_model_small.vocabulary_)

  return bag_of_words_df_small

In [41]:
df_5 = df_range(corpus, 0.8, 2)
df_5.head()

,document,first,is,this
0,1,1,1,1
1,1,0,1,1
2,0,0,0,0
3,1,1,1,1


## 單詞的正則表達式模式

用於控制單詞的分詞規則。默認情況下，token_pattern 的值是 r"(?u)\b\w\w+\b"，表示使用至少包含兩個字母的單詞作為單詞特徵。

In [48]:
def token_pattern(corpus, token_pattern):
  # 濾掉停用詞
  bag_of_words_model_small = CountVectorizer(token_pattern=token_pattern)

  # 統計詞頻轉成dataframe
  bag_of_words_df_small = pd.DataFrame(bag_of_words_model_small.fit_transform(corpus).todense())

  # 加上列名
  bag_of_words_df_small.columns = sorted(bag_of_words_model_small.vocabulary_)

  return bag_of_words_df_small

In [50]:
import re

# 自定義 token_pattern 正則表達式
# 這個正則表達式將匹配至少包含兩個字母或數字的單詞
# 例如：'abc', '123', 'a1b2', 但不匹配單個字母或數字如 'a' 或 '1'
my_token_pattern = r"(?u)\b\w\w+\b|\d+"


df_6 = token_pattern(corpus, my_token_pattern)
df_6.head()


,and,document,first,is,one,second,the,third,this
0,0,1,1,1,0,0,1,0,1
1,0,1,0,1,0,2,1,0,1
2,1,0,0,0,1,0,1,1,0
3,0,1,1,1,0,0,1,0,1


## 自定義函數

可以通過自定義的函數來實現特定的文本處理。比如自定義分詞函數
```
def my_tokenizer(text):

return text.split()

  

vectorizer = CountVectorizer(tokenizer=my_tokenizer)
```


In [45]:
def customizer(corpus, tokenizer):
  # 使用自定義的tokenizer
  bag_of_words_model_small = CountVectorizer(tokenizer=tokenizer)

  # 統計詞頻轉成dataframe
  bag_of_words_df_small = pd.DataFrame(bag_of_words_model_small.fit_transform(corpus).todense())

  # 加上列名
  bag_of_words_df_small.columns = sorted(bag_of_words_model_small.vocabulary_)

  return bag_of_words_df_small

In [47]:
def my_tokenizer(text):
  return text.split()

df_7 = customizer(corpus, my_tokenizer)
df_7.head()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,and,document.,document?,first,is,one.,second,the,third,this
0,0,1,0,1,1,0,0,1,0,1
1,0,1,0,0,1,0,2,1,0,1
2,1,0,0,0,0,1,0,1,1,0
3,0,0,1,1,1,0,0,1,0,1
